%%markdown

# GigaSpaces Task Examples
This notebook will demonstrate the basics of the GigaSpaces task API.



%%markdown

##### GigaSpaces start up instructions

1. In a console window, go to the GigaSpaces install directory/bin (for convenience GS_HOME has already been defined).

`cd $GS_HOME/bin`

2. Start the service grid with a manager and 4 GSCs (in developer mode)

`./gs.sh host run-agent --auto --gsc=4`

3. In another console window, deploy a space named demo.

`./gs.sh space deploy --partitions=2 --ha demo`


In [ ]:
%%pom
repositories:
  - id: org.openspaces
    url: https://maven-repository.openspaces.org
dependencies:
  - groupId: org.gigaspaces
    artifactId: xap-openspaces
    version: 16.4.0

  - groupId: com.gigaspaces.training
    artifactId: model
    version: 1.0
        

In [ ]:
%%classpath

In [ ]:
%%java

import com.gigaspaces.async.AsyncFuture;
import com.gigaspaces.async.AsyncFutureListener;
import com.gigaspaces.async.AsyncResult;


import org.openspaces.core.GigaSpace;
import org.openspaces.core.GigaSpaceConfigurer;
import org.openspaces.core.space.SpaceProxyConfigurer;

import org.openspaces.core.executor.DistributedTask;

import java.util.List;



In [ ]:
%%java

// define the DistributedTask
public class MyDistTask implements DistributedTask<Integer, Long>, java.io.Serializable {
    private static final long serialVersionUID = 0L;
    
    public Integer execute() throws Exception {
        return 1;
    }

    @Override
    public Long reduce(List<AsyncResult<Integer>> results) throws Exception {
        long count = 0;
        for (AsyncResult<Integer> result : results) {
            if (result.getException() != null) {
                throw result.getException();
            }
            count += result.getResult();
        }
        return count;
    }
}


In [ ]:
%%java

public class MyFutureListener implements AsyncFutureListener<Long> {
    @Override
    public void onResult (AsyncResult <Long> asyncResult) {
        if (asyncResult.getException() != null) {
            try {
                throw asyncResult.getException();
            } catch (Exception e) {
                e.printStackTrace();
            }
        }
    }
}

In [ ]:
%%java
// an example of instantiating a remote proxy
SpaceProxyConfigurer configurer = new SpaceProxyConfigurer("demo");
GigaSpace gigaSpace = new GigaSpaceConfigurer(configurer).gigaSpace();

AsyncFuture<Long> future = gigaSpace.execute(new MyDistTask());

future.setListener(new MyFutureListener());

long result = future.get();

System.out.println("Result is: " + result);
